# Demonstrating `mobile-env:smart-city`

`mobile-env` is a simple and open environment for training, testing, and evaluating a decentralized metaverse environment.

* `mobile-env:smart-city` is written in pure Python
* It allows simulating various scenarios with moving users in a cellular network with a single base station and multiple stationary sensors
* `mobile-env:smart-city` implements the standard [Gymnasium](https://gymnasium.farama.org/) (previously [OpenAI Gym](https://gym.openai.com/)) interface such that it can be used with all common frameworks for reinforcement learning
* `mobile-env:smart-city` is not restricted to reinforcement learning approaches but can also be used with conventional control approaches or dummy benchmark algorithms
* It can be configured easily (e.g., adjusting number and movement of users, properties of cells, etc.)
* It is also easy to extend `mobile-env:smart-city`, e.g., implementing different observations, actions, or reward

As such `mobile-env:smart-city` is a simple platform to test RL algorithms in a decentralized metaverse environment.


**Demonstration Steps:**

This demonstration consists of the following steps:

1. Installation and usage of `mobile-env` with dummy actions
2. Configuration of `mobile-env` and adjustment of the observation space (optional)
3. Training a single-agent reinforcement learning approach with [`stable-baselines3`](https://github.com/DLR-RM/stable-baselines3)

In [1]:
# First, install stable baselines; only SB3 v2.0.0+ supports Gymnasium
%pip install stable-baselines3==2.0.0 tensorboard

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing necessary libraries
import gymnasium
import mobile_env
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_checker import check_env

# predefined small scenarios
from mobile_env.scenarios.smart_city import MComSmartCity

# easy access to the default configuration
MComSmartCity.default_config()

/Users/elifohri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'width': 200,
 'height': 200,
 'EP_MAX_TIME': 100,
 'seed': 666,
 'reset_rng_episode': False,
 'arrival': mobile_env.core.arrival.NoDeparture,
 'channel': mobile_env.core.channels.OkumuraHata,
 'scheduler': mobile_env.core.schedules.ResourceFair,
 'movement': mobile_env.core.movement.RandomWaypointMovement,
 'utility': mobile_env.core.utilities.BoundedLogUtility,
 'handler': mobile_env.handlers.smart_city_handler.MComSmartCityHandler,
 'bs': {'bw': 100000000.0,
  'freq': 2500,
  'tx': 40,
  'height': 50,
  'computational_power': 100},
 'ue': {'velocity': 1.5, 'snr_tr': 2e-08, 'noise': 1e-09, 'height': 1.5},
 'sensor': {'height': 1.5, 'snr_tr': 2e-08, 'noise': 1e-09},
 'ue_job': {'job_generation_probability': 0.7,
  'communication_job_lambda_value': 10.0,
  'computation_job_lambda_value': 10.0},
 'sensor_job': {'communication_job_lambda_value': 5.0,
  'computation_job_lambda_value': 5.0},
 'e2e_delay_threshold': 3.0,
 'reward_calculation': {'ue_penalty': -1.0,
  'discount_factor': 0.95

In [3]:
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
import gymnasium as gym

env = gym.make("mobile-smart_city-smart_city_handler-v0", render_mode="rgb_array")

# train PPO agent on environment. this takes a while
model = PPO(MlpPolicy, env, tensorboard_log='results_sb', verbose=1)
model.learn(total_timesteps=100000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to results_sb/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | 97.1     |
| time/              |          |
|    fps             | 98       |
|    iterations      | 1        |
|    time_elapsed    | 20       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | 111          |
| time/                   |              |
|    fps                  | 98           |
|    iterations           | 2            |
|    time_elapsed         | 41           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0070546167 |
|    clip_fraction        | 0.0884       |
|    clip_range       

In [4]:
from datetime import datetime

# Save the trained model
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")  # Add timestamp to avoid overwriting
model.save(f"ppo_smart_city_env_model_{timestamp}")


In [5]:
# show training progress
%load_ext tensorboard
%tensorboard --logdir results_sb

In [6]:
print(f"Action space: {env.action_space}")

Action space: Box(0.0, 1.0, (2,), float32)


In [7]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=False)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")


/Users/elifohri/Library/Python/3.9/lib/python/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: 1773.6801096990705, Std reward: 519.7837539019971
